# Chance-constrained programming: portfolio optimization example

In [18]:
# ENV["GUROBI_HOME"] = "/Library/gurobi952/macos_universal2/"
# # import Pkg
# Pkg.add("Gurobi")
# Pkg.build("Gurobi")
# Pkg.add("Dates")
# Pkg.add("CSV")
# Pkg.add("DataFrames")

   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`
⌃ [2e9cd046] + Gurobi v0.11.3
    Updating `~/.julia/environments/v1.9/Manifest.toml`
⌃

 [2e9cd046] + Gurobi v0.11.3
        Info Packages marked with ⌃ have new versions available and may be upgradable.


    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/87c018cbd2fd33e6d2462d486abee53a27c91c91/build.log`


In [32]:
using LinearAlgebra
using JuMP
using HiGHS, Gurobi
using Distributions
using Random
using CSV, DataFramesMeta
using Dates
using Statistics

## HW2

Sample Average Approximation Method for Chance Constrained Programming : Theory and Applications”, 2009, Section 3, en s ́electionnant 10 actifs depuis la base de donn ́ees disponible sur Kaggle `a l’adresse https: //www.kaggle.com/datasets/jacksoncrow/stock-market-dataset.

Note : vous pouvez partir de https://www.kaggle.com/code/artemburenok/ stock-analysis-monte-carlo-build-portfolio pour analyser les donn ́ees.

Adapted from Stephen Boyd and Lieven Vandenberghe, "Convex Optimization", Cambridge University Press, 2004, Section 4.7.6, p. 187.

We consider an investiment portfolio with $n$ assets and random returns, except the last one. We assume that the $n-1$ first returns follow a multivariate distribution with known mean and covariance matrix. The last asset is a risk-free investment product, with a fixed return.

The assets characteristics are detailed below.

In [13]:
company_list = ["MMM", "AMZN", "CAT", "AI", "AIR", "AM", "TWTR", "RIO", "MSFT", "BP"]


Dict{Any, Any}()

In [15]:
dir_path = "../data/stocks/"
companies = Dict{String, DataFrame}()
for company in company_list
    fn = dir_path * company * ".csv"
    companies[company] = CSV.read(fn, DataFrame, select=["Date", "Adj Close"])
end

In [19]:
# If Initialize to same start and end date  
first_end, last_start = DateTime(2023, 11, 14), DateTime(1900, 11, 14)
for (company, df) in companies

    start_date = minimum(df.Date)
    end_date = maximum(df.Date)

    println("$(company): $(start_date) to $(end_date)")
    if start_date > last_start
        last_start = start_date
        println("last_start: $(start_date)")
    end
    if end_date < first_end
        first_end = end_date
        println("first_end: $(end_date)")
    end
end

for (company, df) in companies
    df = df[df.Date .>= last_start, :]
    df = df[df.Date .<= first_end, :]
    companies[company] = df
end

In [20]:
# Calculate returns
for (company, df) in companies
    df[!, :Return] = [0.0; diff(df[!, :"Adj Close"]) ./ df[1:end-1, :"Adj Close"]]
    companies[company] = df
end

In [34]:
# Calculate mean and covariance
mean_return = zeros(length(company_list))
covariance = zeros(length(company_list), length(company_list))

df = companies["MMM"]
println("mean: $(mean(df[!, :Return]))")

In [35]:
println("hello")

In [33]:
# Calculate mean and covariance
mean_return = zeros(length(company_list))
covariance = zeros(length(company_list), length(company_list))
# for (company, df) in companies

#     # println("company $(mean(companies[company][!, :Return]))")
#     println("coompany")
#     # mean_return[i] = mean(companies[company][!, :Return])
#     # for (j, company2) in enumerate(company_list)
#     #     covariance[i, j] = cov(companies[company][!, :Return], companies[company2][!, :Return])
#     # end
# end
df = companies["MMM"]
println("mean: $(mean(df[!, :Return]))")

In [26]:
n = 4  # the last asset is a risk-free asset, with a null variance.
μ = [.12 ; .10 ; .07 ; .03]
Σ = [ 4e-2  6e-3 -4e-3  ;
      6e-3  1e-2  0.0  ;
      -4e-3  0.0 2.5e-3 ]

# We create une multivariate normal of mean μ and covariance matrix Σ
d = MvNormal(μ[1:n-1], Σ)

FullNormal(
dim: 3
μ: [0.12, 0.1, 0.07]
Σ: [0.04 0.006 -0.004; 0.006 0.01 0.0; -0.004 0.0 0.0025]
)


In [28]:
# Estimate the probability to have a negative return and the resulting expected shortfall.
function expectedshortfall(p:: Vector, d:: Distribution, M:: Int = 1000000)
    
    loss = 0
    vloss = 0
    for i = 1:M
        ξ = [rand(d); μ[n]]
        ret = dot(p, ξ)
        if ret < 0
            loss += 1
            vloss += ret
        end
    end

    return loss/M, vloss/loss
    
end

expectedshortfall (generic function with 2 methods)

## Portfolio with uniform repartition

We first consider the naive strategy where the same amount is invested in each asset.

In [35]:
# Expected return with a uniform repartition.
p = ones(n)./n
er = sum(p[i]*μ[i] for i = 1:n)

0.08000000000000002

We now compute the loss probability and the average loss when a loss occurs.

In [36]:
expectedshortfall(p, d)

(0.089571, -0.027525206361675204)

## Optimal decision without loss constraint

We now aim to maximize the expected return, without any consideration for the potential loss.

In [37]:
m = Model(HiGHS.Optimizer)

@variable(m, p[1:n] >= -0.1)
@constraint(m, sum(p[i] for i = 1:n) <= 1)

@objective(m, Max, sum(p[i]*μ[i] for i = 1:n))

println(m)

Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 p[1] + p[2] + p[3] + p[4] ≤ 1.0
 p[1] ≥ -0.1
 p[2] ≥ -0.1
 p[3] ≥ -0.1
 p[4] ≥ -0.1



In [43]:
optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xcc387da0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e-01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 4 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.360000000e-01

User-callback calls 34, time in user-callback 0.02 sec


In [44]:
value.(p)

4-element Vector{Float64}:
  1.3000000000000003
 -0.1
 -0.1
 -0.1

Not surprisingly, we invest everything is the asset having the highest return rate, even borrowing from the lower return rate asset.

In [32]:
objective_value(m)

0.13599999999999998

The loss probability is however close to 30%, and the average lost amount is significantly more important if a loss occurs.

In [45]:
expectedshortfall(value.(p), d)

(0.300211, -0.16420566406226844)

## Optimal decision with loss constraint

We add the constraint that we want to limit the risk by accepting a loss with a maximum probability of 0.05.

In [46]:
maxloss = 0

α = 0.95
z = 1/quantile(Normal(0,1), α)

0.6079568319117693

We build the second-order cone constraint corresponding to the joint chance constraint.

In [34]:
A = Σ^0.5

3×3 Symmetric{Float64, Matrix{Float64}}:
  0.198273   0.0203895   -0.0164913
  0.0203895  0.0978718    0.00231869
 -0.0164913  0.00231869   0.0471451

In [41]:
A*A-Σ

3×3 Matrix{Float64}:
 -6.93889e-18   5.20417e-18   1.73472e-18
  5.20417e-18  -1.73472e-18  -1.22964e-18
  1.73472e-18  -1.22964e-18   0.0

Unfortunately, HiGHS does not support second-order cone constraint. We switch to Gurobi.

In [47]:
set_optimizer(m, Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13


In [48]:
# || x || <= t, t >= 0
# https://jump.dev/JuMP.jl/stable/reference/constraints/#JuMP.SecondOrderCone
@constraint(m, [z*(-maxloss+sum(μ[i]*p[i] for i = 1:n)); (Σ^0.5)*p[1:n-1]] in SecondOrderCone())

println(m)

Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 

p[1] + p[2] + p[3] + p[4] ≤ 1.0
 [0.0729548198294123 p[1] + 0.06079568319117693 p[2] + 0.04255697823382386 p[3] + 0.018238704957353077 p[4], 0.19827331129330036 p[1] + 0.02038945638178348 p[2] - 0.016491334004420634 p[3], 0.02038945638178348 p[1] + 0.09787182302877327 p[2] + 0.002318690466585327 p[3], -0.016491334004420634 p[1] + 0.002318690466585327 p[2] + 0.04714509070173485 p[3]] ∈ MathOptInterface.SecondOrderCone(4)
 p[1] ≥ -0.1
 p[2] ≥ -0.1
 p[3] ≥ -0.1
 p[4] ≥ -0.1



In [49]:
optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 5 rows, 8 columns and 21 nonzeros
Model fingerprint: 0x9ff28c3e
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e-02, 1e-01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 5 rows, 8 columns, 21 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.11301574e-01  5.68875345e-02  3.97e-01 1.28e-01  3.28e-02     0s
   1   9.26826200e-02  1.04042953e-01  2.41e-02 1.64e-02  6.54e-03     

In [ ]:
sol = value.(p)

In [ ]:
sol = value.(p)

In [ ]:
objective_value(m)

Without any surprise, the expected return is less than without the loss constraint, but is still higher than with the uniform repartition. We can also see that we use the risk-free asset as a borrowing tool. The risk to lose money is limited to 5%, as desired, and is less than any other strategy. Interestingly, the average loss is also the smallest one.

In [ ]:
expectedshortfall(value.(p), d)